In [1]:
# Get autocomplete to work
%config Completer.use_jedi = False

# Ensure external Python files are refreshed when reimporting things
%load_ext autoreload
%autoreload 2

In [5]:
from load_functions import load_text
    
text_input = load_text(day=4)
print(len(text_input))
text_input[:13]

601


['31,50,79,59,39,53,58,95,92,55,40,97,81,22,69,26,6,23,3,29,83,48,18,75,47,49,62,45,35,34,1,88,54,16,56,77,28,94,52,15,0,87,93,90,60,67,68,85,80,51,20,96,61,66,63,91,8,99,70,13,71,17,7,38,44,43,5,25,72,2,57,33,82,78,89,21,30,11,73,84,4,46,14,19,12,10,42,32,64,98,9,74,86,27,24,65,37,41,76,36',
 '',
 '31  5 70  8 88',
 '38 63 14 91 56',
 '22 67 17 47 74',
 '93 52 69 29 53',
 '33 66 64 19 73',
 '',
 '35 63 17 48 77',
 '25 58 33 14 96',
 '32 87 90 66 70',
 '16  4 98 72 23',
 '19 74 39 29 59']

#### Turn into number draw, and bingo boards

In [89]:
import pandas as pd

numbers_drawn = [int(n) for n in text_input[0].split(',')]

# Boards are present in groups of 6 lines - 1 blank followed by 5 rows of numbers
num_boards = int((len(text_input) - 1) / 6)

# Turn bingo boards into list of dataframes
board_list = []
for i in range(num_boards):
    
    # Starting from index 2, each board is spaced 6 lines apart
    ind_start = i*6 + 2
    ind_end = ind_start + 5 # List slices exclude end index
    
    # Get 1x5 list of bingo numbers
    board_flat = text_input[ind_start:ind_end]
    
    # Convert to 5x5 list
    board_split = [t.split() for t in board_flat]
    
    board = pd.DataFrame(data=board_split,
                         columns=['col'+str(i) for i in range(1, 6)],
                         index=['row'+str(i) for i in range(1, 6)]).astype(int)
    
    board_list.append(board)
    
board_list[1]

,col1,col2,col3,col4,col5
row1,35,63,17,48,77
row2,25,58,33,14,96
row3,32,87,90,66,70
row4,16,4,98,72,23
row5,19,74,39,29,59


# Problem 

#### Part 1: play bingo, find which board wins (complete row or column), then multiply last number called with the remaining unstruck numbers on the winning board

In [101]:
import copy
board_list_pt1 = copy.deepcopy(board_list)

def check_number(board, num):
    
    # Given a bingo number, check if it exists on a board and if so - strike it off
    for row in range(5):
        for col in range(5):
            if board.iloc[row, col] == num:
                board.iloc[row, col] = 'X'

    
def check_win(df_line, board_ind, print_board):
    
    # Check whether there are 5 'X's in a dataframe row/column
    n_xs = 0
    
    for v in df_line.values:
        if v == 'X':
            n_xs += 1
    
    if n_xs == 5:
        if print_board:
            print(f'Winning board: {board_ind}')
        return True


def check_board_win(board, board_ind, print_board=True):
    
    # Check that a bingo board has an entire row or column complete
    for row in range(5):
        if check_win(board.iloc[row, :], board_ind, print_board):
            return True
    
    for col in range(5):
        if check_win(board.iloc[:, col], board_ind, print_board):
            return True

        
def board_sum(board):
    
    # When a board wins, provide a sum of all remaining non-struck values
    result = 0
    
    # Flatten board to list
    board_values = board.values.flatten()
    
    # Sum all numeric values in a board
    for v in board_values:
        result += (0 if v == 'X' else v)
    
    return result
            
    
# Loop through numbers drawn
game_won = False

for n in numbers_drawn:
    
    # Loop through boards
    for (i, b) in enumerate(board_list_pt1):
        
        # Check number on board, cross off if it exists
        check_number(board=b, num=n)
        
        # Check whether board has won, if so, sum board
        if check_board_win(board=b, board_ind=i):
            print(b, f'Number: {n}', sep='\n')
            print(f'Board sum: {board_sum(board=b)}')
            print('Day 4, part 1:', board_sum(board=b) * n)
            game_won = True
            break
        
    if game_won:
        break

Winning board: 53
     col1 col2 col3 col4 col5
row1   41    X   44    2    X
row2   38   14   19   72   64
row3    X    X    X    X    X
row4   70    7    1    X   86
row5    X    X   90   96   82
Number: 35
Board sum: 726
Day 4, part 1: 25410


#### Part 2: find the final board to win

In [144]:
board_list_pt2 = copy.deepcopy(board_list)

# Loop through numbers drawn
board_indices = [i for i in range(len(board_list_pt2))]
game_won = False

for n in numbers_drawn:
    
    # Loop through boards
    for (i, b) in enumerate(board_list_pt2):

        # Check number on board, cross off if it exists
        check_number(board=b, num=n)

        # Only check for wins for board yet to win
        if i in board_indices:

            # Check whether board has won, if so, sum board
            if check_board_win(board=b, board_ind=i, print_board=False):

                # Check if this is the last board to win out of all boards, if so, end game
                if len(board_indices) == 1:
                    print(b, f'Number: {n}', sep='\n')
                    print(f'Board sum for index {i}: {board_sum(board=b)}')
                    print('Day 4, part 2:', board_sum(board=b) * n)
                    game_won = True

                board_indices.remove(i)
        
    if game_won:
        break

     col1 col2 col3 col4 col5
row1   24    X    X    X    X
row2    X    X    9    X   27
row3    X   37    X    X    X
row4    X    X    X    X    X
row5    X    X   98    X    X
Number: 14
Board sum for index 40: 195
Day 4, part 2: 2730
